In [1]:
import pandas as pd
import numpy as np
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-11 23:53:00,0.43945,0.0,0.0,0.0,2022-06-11,23:53:00,23
1,2022-06-11 23:54:00,0.48828,0.0,0.0,0.0,2022-06-11,23:54:00,23
2,2022-06-11 23:55:00,0.46387,0.0,0.0,0.0,2022-06-11,23:55:00,23
3,2022-06-11 23:56:00,0.46387,0.0,0.0,0.0,2022-06-11,23:56:00,23
4,2022-06-11 23:57:00,0.46387,0.0,0.0,0.0,2022-06-11,23:57:00,23


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
15189,2022-06-22 22:23:00,2022-06-22,22:23:00,22,0.80566,0.0,0.0,0.0
15190,2022-06-22 22:24:00,2022-06-22,22:24:00,22,0.73242,0.0,0.0,0.0
15191,2022-06-22 23:24:00,2022-06-22,23:24:00,23,0.73242,0.0,0.0,0.0
15192,2022-06-23 00:06:00,2022-06-23,00:06:00,0,0.21973,0.0,0.0,0.0
15193,2022-06-23 00:07:00,2022-06-23,00:07:00,0,0.24414,0.0,0.0,0.0


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-11T00:00:00.000000000', '2022-06-12T00:00:00.000000000',
       '2022-06-13T00:00:00.000000000', '2022-06-14T00:00:00.000000000',
       '2022-06-15T00:00:00.000000000', '2022-06-16T00:00:00.000000000',
       '2022-06-17T00:00:00.000000000', '2022-06-18T00:00:00.000000000',
       '2022-06-19T00:00:00.000000000', '2022-06-20T00:00:00.000000000',
       '2022-06-21T00:00:00.000000000', '2022-06-22T00:00:00.000000000',
       '2022-06-23T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

In [7]:
filter_daily_values = df[(df['Date'] >= '2022-06-18') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
8169,2022-06-18,0,0.0
8170,2022-06-18,0,0.0
8171,2022-06-18,0,0.0
8172,2022-06-18,0,0.0
8173,2022-06-18,0,0.0
...,...,...,...
15187,2022-06-22,22,0.0
15188,2022-06-22,22,0.0
15189,2022-06-22,22,0.0
15190,2022-06-22,22,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values.head()

,Date,Hour,Power (KW)
0,2022-06-18,0,0.0
1,2022-06-18,1,0.0
2,2022-06-18,2,0.0
3,2022-06-18,3,0.0
4,2022-06-18,4,0.0


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data.drop(['RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
weather_data.tail()

,SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
127,49.4,16.4,77,1
128,61.0,18.5,68,11
129,73.3,20.4,60,11
130,82.9,21.9,53,11
131,80.4,23.5,45,25


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(14)

,Power (KW),SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
118,0.0,0.0,0.0,0,0
119,0.0,0.0,0.0,0,0
120,NaN,0.0,0.0,0,0
121,NaN,0.0,0.0,0,0
122,NaN,0.0,0.0,0,0
123,NaN,0.0,0.0,0,0
124,NaN,0.0,0.0,0,0
125,NaN,8.4,12.9,85,0
126,NaN,30.5,14.2,85,0
127,NaN,49.4,16.4,77,1


In [12]:
count_total_rows = len(df1) - 12

In [13]:
independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]

In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
reg = linear_model.LinearRegression(fit_intercept = False)
reg.fit(independent_columns, dependent_column)

LinearRegression(fit_intercept=False)

In [16]:
forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(12)

In [17]:
return_array = list(reg.predict(forcasted_data))

In [18]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [19]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [20]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [21]:
data_dict = {'Date': current_date_12, 'Hour': hours_12, 'Power (KW)': return_array}

In [22]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-06-23,0,0.000000
1,2022-06-23,1,0.000000
2,2022-06-23,2,0.000000
3,2022-06-23,3,0.000000
4,2022-06-23,4,0.000000
5,2022-06-23,5,0.541865
6,2022-06-23,6,0.980895
7,2022-06-23,7,1.408004
8,2022-06-23,8,1.632652
9,2022-06-23,9,1.940484


In [23]:
# dependent_variable = 'Power (KW)'

In [24]:
# independent_variables = df1.columns.tolist()
# independent_variables.remove(dependent_variable)

In [25]:
# independent_variables

In [26]:
# X = df1[independent_variables].values

In [27]:
# y = df1[dependent_variable].values

In [28]:
# X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [29]:
# X_train

In [30]:
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

In [31]:
# y_pred = regressor.predict(X_test)
# y_pred

In [32]:
# math.sqrt(mean_squared_error(y_test, y_pred))

In [33]:
# r2_score(y_test, y_pred)